In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import plotly.express as px
from sales_forecasting.api.model import RequestBody
from pydantic import parse_obj_as
from typing import List
import gc

In [3]:
raw_sales_data=pd.read_csv('..//data//sales_per_store_per_day.csv', dtype={'Store':'Int64','DayOfWeek':'Int64','Sales':'Int64','Customers':'Int64','Open':'Int64','Promo':'Int64','StateHoliday':'str','SchoolHoliday':'Int64'}, parse_dates=['Date'])
raw_dict=raw_sales_data.to_dict(orient='records')

validated_sales=parse_obj_as(List[RequestBody],raw_dict)
del raw_sales_data, raw_dict
gc.collect()

0

In [4]:
sales_df=pd.DataFrame([{
    key:value 
    for key, value in validated_sale.dict().items()
}
for validated_sale in validated_sales
])

del validated_sales 
gc.collect()


0

In [5]:
sales_df['sales_date']=pd.to_datetime(sales_df['sales_date'])

In [6]:
print('Minim available data info is from:',sales_df['sales_date'].min())
print('Maxim available data info is from:',sales_df['sales_date'].max())
sales_df['year']=sales_df['sales_date'].dt.year

Minim available data info is from: 2013-01-01 00:00:00
Maxim available data info is from: 2015-06-30 00:00:00


In [7]:
print('Store data appearance')
sales_df['store'].value_counts()

Store data appearance


233     911
836     911
308     911
309     911
821     911
       ... 
778     727
677     727
952     727
1065    727
512     727
Name: store, Length: 1115, dtype: int64

In [ ]:
##customers=no of customers per day at one store
##sales = total sales amount at one store per day
## state_holiday:
## - b=Easter
## - c=Christmas
## - a=other holidays
## data from same country?
## magazinele nu au acelasi nr de inregistrari

## Holidays analysis

In [58]:
import holidays
import datetime
us_holidays = holidays.UnitedStates()

In [39]:
sales_holidays=set(sales_df[sales_df['state_holiday']!='0']['sales_date'])

In [52]:
sales_df.dtypes

sales_date        datetime64[ns]
store                      int64
sales                      int64
customers                  int64
open_flag                  int64
promo                      int64
day_of_week                int64
state_holiday             object
school_holiday             int64
year                       int64
dtype: object

In [78]:
sales_df[sales_df['sales_date']==datetime.datetime(2014,12,25)]['open_flag'].value_counts()

0    919
1     16
Name: open_flag, dtype: int64

In [59]:
set([sales for sales in sales_holidays
if sales not in us_holidays])

{Timestamp('2013-01-06 00:00:00'),
 Timestamp('2013-03-29 00:00:00'),
 Timestamp('2013-04-01 00:00:00'),
 Timestamp('2013-05-01 00:00:00'),
 Timestamp('2013-05-09 00:00:00'),
 Timestamp('2013-05-20 00:00:00'),
 Timestamp('2013-05-30 00:00:00'),
 Timestamp('2013-08-15 00:00:00'),
 Timestamp('2013-10-03 00:00:00'),
 Timestamp('2013-10-31 00:00:00'),
 Timestamp('2013-11-01 00:00:00'),
 Timestamp('2013-11-20 00:00:00'),
 Timestamp('2013-12-26 00:00:00'),
 Timestamp('2014-01-06 00:00:00'),
 Timestamp('2014-04-18 00:00:00'),
 Timestamp('2014-04-21 00:00:00'),
 Timestamp('2014-05-01 00:00:00'),
 Timestamp('2014-05-29 00:00:00'),
 Timestamp('2014-06-09 00:00:00'),
 Timestamp('2014-06-19 00:00:00'),
 Timestamp('2014-10-03 00:00:00'),
 Timestamp('2014-10-31 00:00:00'),
 Timestamp('2014-11-01 00:00:00'),
 Timestamp('2014-11-19 00:00:00'),
 Timestamp('2014-12-26 00:00:00'),
 Timestamp('2015-01-06 00:00:00'),
 Timestamp('2015-04-03 00:00:00'),
 Timestamp('2015-04-06 00:00:00'),
 Timestamp('2015-05-

## Nonstop available stores

In [ ]:
## top magazine

In [84]:
stores_per_open_holiday=sales_df.groupby(['store','open_flag','state_holiday','school_holiday']).agg({
    'customers':'sum',
    'sales':'sum'
})
stores_per_open_holiday.reset_index(inplace=True)

In [97]:
nonstop_stores=stores_per_open_holiday[(stores_per_open_holiday['state_holiday']!='0') & (stores_per_open_holiday['open_flag']==1)].copy()

In [100]:
nonstop_stores['sales/customers']=nonstop_stores['sales']/nonstop_stores['customers']

In [106]:
len(nonstop_stores['store'].unique())

156

In [101]:
import plotly.express as px
fig = px.bar(nonstop_stores, x='store', y='sales/customers')
fig.show()

In [102]:
nonstop_stores

,store,open_flag,state_holiday,school_holiday,customers,sales,sales/customers
17,2,1,a,0,770,5345,6.941558
39,5,1,a,0,1775,16179,9.114930
47,6,1,a,0,1977,17516,8.859889
55,7,1,a,0,889,6355,7.148481
398,54,1,a,0,2484,21811,8.780596
...,...,...,...,...,...,...,...
8149,1099,1,a,0,6309,39991,6.338722
8150,1099,1,b,1,2590,15666,6.048649
8151,1099,1,c,1,1191,7648,6.421495
8210,1107,1,a,1,649,6138,9.457627


In [17]:
#analyse unsuccesful stores --- with less customers
#analyse the most profitable days
# efectele vacantelor 
# evolutia individuala a magazinelor
# outlieri
#magazinele nu sunt inchise in acelasi timp in corelatie cu vacanta

,sale_date,store,sales,customers,open_flag,promo,day_of_week,state_holiday,school_holiday
0,2015-06-30,1,5735,568,1,1,2,0,0
1,2015-06-30,2,9863,877,1,1,2,0,0
2,2015-06-30,3,13261,1072,1,1,2,0,1
3,2015-06-30,4,13106,1488,1,1,2,0,0
4,2015-06-30,5,6635,645,1,1,2,0,0
...,...,...,...,...,...,...,...,...,...
982639,2013-01-01,1111,0,0,0,0,2,a,1
982640,2013-01-01,1112,0,0,0,0,2,a,1
982641,2013-01-01,1113,0,0,0,0,2,a,1
982642,2013-01-01,1114,0,0,0,0,2,a,1
